<a href="https://colab.research.google.com/github/naeemsaleem2003/EliteTennisAcademyWeb/blob/main/hw1_Naeem_Saleem_(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EECS 487 HW 1 Coding (C) Section: Language Model and Naive Bayes Classifier

This notebook contains the coding (C) section of HW 1. In the first problem, you will build two n-gram language models for Yelp reviews, one that operates at the word-level and another that operates on the level of individual characters. You will then use these language models to generate sample Yelp reviews. In the second problem, you will build naive bayes classifiers to distinguish between legitimate news headlines and clickbait.

After this assignment, you will learn to
1. train ngram language models given a text corpus;
2. generate text from a language model;
3. calculate probability of some text given a language model;
4. classify news headlines using naive bayes classifiers;
5. evaluate classifiers by calculating the performance on test set.

**Do not edit anything in this python notebook.** All the code you need to write are in ```language_model.py``` and ```naive_bayes.py```.

## Setup
Run the following cell to upgrade your NLTK version to the latest. You should be using NLTK version >= 3.8 for proper results!!

In [1]:
!# pip install nltk --upgrade  # after running this line once, you can comment this line out
import nltk
nltk.download('punkt_tab')
print(nltk.__version__) # make sure this is >=3.8

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


3.9.1


Before we get started, run the following cell to load the autoreload extension so that functions in ```language_model.py``` and ```naive_bayes.py``` will be re-imported into the notebook every time we run them. We also need to import all necessary packages.

In [3]:
%reload_ext autoreload
%autoreload 2

import pickle

import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

from language_model import *
from naive_bayes import *

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## C.1 N-gram Language Model [40 points]
In this problem, you will train two language models on Yelp reviews, one word-level (up to trigrams) and the other character-level (up to 4-grams). The [dataset provided](https://www.kaggle.com/datasets/omkarsabnis/yelp-reviews-dataset?resource=download) is a subset of [a larger dataset](https://www.yelp.com/dataset) published by Yelp. We downloaded the file ```yelp.csv``` for you. To begin, you need to first load the data. Here we provide the code for you, but take a look at how we do it because you will need to load the data by yourself later.

In [5]:
filename = 'yelp.csv'
df = pd.read_csv(filename)

all_text = df['text']

trn_text, dev_text = train_test_split(all_text, test_size=0.2, random_state=42)
trn_text, dev_text = trn_text.reset_index(drop=True), dev_text.reset_index(drop=True)
print("trn_text:")
print(trn_text)
print("\ndev_text:")
print(dev_text)

trn_text:
0       Consistent with the east coast stores, great f...
1       I had a serious craving for Roti.  So glad I f...
2       Cool atmosphere and good service.\nThe food is...
3       I loved this place! Amazing food and service. ...
4       This is the pub burger you have been looking f...
                              ...                        
7995    Very relaxed atmosphere and employees are supe...
7996    07/25/11\n\nI have a new crush... and it's Chr...
7997    I've been frequenting the Dollar store for the...
7998    I love the girls there. I get my eyebrows done...
7999    Scale of 1-10 (multiple visits):\n10 Food\n9 S...
Name: text, Length: 8000, dtype: object

dev_text:
0       We got here around midnight last Friday... the...
1       Brought a friend from Louisiana here.  She say...
2       Every friday, my dad and I eat here. We order ...
3       My husband and I were really, really disappoin...
4       Love this place!  Was in phoenix 3 weeks for w...
           

### C.1.1 Data processing and n-gram counts [10 points]
Now you need to train your language models on these reviews. You need to implement the class ```NGramLM```. First, **fill in** the function ```get_ngram_counts``` to process the reviews and get the counts of all n-grams in the reviews.

Program your `NGramLM` class and `get_ngram_counts` function to collect n-grams of size `n <= ngram_size`. If you refer to the cells below, you will see that `ngram_size = 3` for the word-level model (so collect unigrams, bigrams, and trigrams) and `ngram_size = 4` for the character-level model (so collect unigrams, bigrams, trigrams, and four-grams).

Store these n-grams in the dictionary `self.ngram_count`. This dictionary will contain other dictionaries as values. For instance, `self.ngram_count[0]` will be a dictionary containing all of the unigrams, and `self.ngram_count[1]` will be a dictionary containing all the bigrams. To access the count of a unigram, simply use it as a key in the unigram dictionary: ```self.ngram_count[0]["word1"]``` will be $C(word1)$. To access the count of a bigram (or trigram or four-gram), simply use a tuple: ```self.ngram_count[1][("word1", "word2")]``` will be $C(word1, word2)$.

Use the following rules when processing the reviews:
- Prepend **two/three** &lt;s&gt; at the beginning of each review as BOS tokens (two for trigram model and three for four-gram model), and append one &lt;/s&gt; at the end of a review as the EOS token.
- Convert all letters to lowercase.
- Tokenize each review. Use `nltk.tokenize.word_tokenize` for the word-level model and `char_tokenizer` (defined below) for the character-level model. For your convenience, each is passed into NGramLM, so you can program the internals of NGramLM to simply utilize whichever tokenizer is passed in. Do not split BOS and EOS tokens (i.e., even in the character-level model, `<s>` will be a single token rather than three tokens).
- Replace all unigrams that occur < 2 times with "UNK". For the character-level model, this means that all characters occuring < 2 times should be replaced with UNK. **NOTE:** do this before collecting (n>1)-grams. For instance, if the word "apple" appeared < 2 times, then the bigram ("red", "apple") should instead be ("red", "UNK").
- You will need to make the function ```get_ngram_counts``` flexible enough so that (1) it can operate on both the character-level and the word-level and (2) it can operate on a variety of maximum ngram sizes (as determined by `ngram_size`)
- Do **NOT** remove punctuation.

Hint: ```collections.defaultdict``` is useful.

In [6]:
bos_token = "<s>"
eos_token = "</s>"

class NGramLM:
    """N-gram language model."""

    def __init__(self, bos_token, eos_token, tokenizer, ngram_size):
        self.ngram_count = {i: defaultdict(int) for i in range(ngram_size)}
        self.ngram_size = ngram_size
        self.vocab_sum = None  # Could be useful in linear interpolation
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.tokenizer = tokenizer

    def get_ngram_counts(self, reviews):
      unigram_counts = defaultdict(int)

      processed_reviews = []
      for review in reviews:
          tokens = self.tokenizer(review.lower())  # Tokenize and lowercase
          tokens = [self.bos_token] * (self.ngram_size - 1) + tokens + [self.eos_token]  # Add BOS/EOS
          processed_reviews.append(tokens)

          # Count unigrams
          for token in tokens:
              unigram_counts[token] += 1

      # Replace rare unigrams (<2 occurrences) with "UNK"
      for i, tokens in enumerate(processed_reviews):
          processed_reviews[i] = [token if unigram_counts[token] >= 2 else "UNK" for token in tokens]

      # Second pass: Count n-grams
      for tokens in processed_reviews:
          for n in range(self.ngram_size):
              for i in range(len(tokens) - n):
                  ngram = tuple(tokens[i:i + n + 1]) if n > 0 else tokens[i]  # Use tuple for bigram and above
                  self.ngram_count[n][ngram] += 1


Now that you have tested ```get_ngram_counts``` on the word-level model (above), test it on the character-level model below.

In [7]:
def char_tokenizer(text):
    return list(text)

bos_token = "<s>"
eos_token = "</s>"
ngram_size = 4
char_lm = NGramLM(bos_token, eos_token, char_tokenizer, ngram_size)

# Sample text
trn_text = ["This is a test.", "I love coding.", "NLP is fun!"]
char_lm.get_ngram_counts(trn_text)
print(f"Number of unigrams: {len(char_lm.ngram_count[0])}")
least_unigram = min(char_lm.ngram_count[0].keys(), key=lambda x: char_lm.ngram_count[0][x])
print(f"Unigram with smallest count: {least_unigram}\tCount: {char_lm.ngram_count[0][least_unigram]}")
print(f"Unknown unigram count: {char_lm.ngram_count[0]['UNK']}")
print(f"Number of BOS token: {char_lm.ngram_count[0][bos_token]}")
print(f"Number of bigrams: {len(char_lm.ngram_count[1])}")

Number of unigrams: 12
Unigram with smallest count: e	Count: 2
Unknown unigram count: 10
Number of BOS token: 9
Number of bigrams: 32


### C.1.2 Add-k Smoothing [5 points]
As discussed in the lecture, simply counting the number of occurrence of n-grams will assign 0 probability to n-grams that don't appear in the training corpus and thus cannot generalize to unseen data. To mitigate this, you need to implement some smoothing techniques. **Fill in** the function ```add_k_prob``` so that given a bigram $(w_1, w_2)$, a unigram $w_3$, and $k$, return $p(w_3|(w_1, w_2))$ after applying add-k smoothing.<br>

Notes:
- Program this flexibly enough so that, for the character-level model, the model can smooth over trigrams and unigrams rather than bigrams and unigrams.
- &lt;s&gt; should **NOT** be considered when calculating the vocabulary size because it will never be generated by the language model (although it's in ```self.unigram_count```). &lt;/s&gt; and "UNK" should both be treated as tokens in the vocabulary.

In [8]:
bos_token = "<s>"
eos_token = "</s>"

class NGramLM:
    """N-gram language model."""

    def __init__(self, bos_token, eos_token, tokenizer, ngram_size):
        self.ngram_count = {i: defaultdict(int) for i in range(ngram_size)}
        self.ngram_size = ngram_size
        self.vocab_sum = None  # Could be useful in linear interpolation
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.tokenizer = tokenizer

    def get_ngram_counts(self, reviews):
      unigram_counts = defaultdict(int)

      processed_reviews = []
      for review in reviews:
          tokens = self.tokenizer(review.lower())  # Tokenize and lowercase
          tokens = [self.bos_token] * (self.ngram_size - 1) + tokens + [self.eos_token]  # Add BOS/EOS
          processed_reviews.append(tokens)

          # Count unigrams
          for token in tokens:
              unigram_counts[token] += 1

      # Replace rare unigrams (<2 occurrences) with "UNK"
      for i, tokens in enumerate(processed_reviews):
          processed_reviews[i] = [token if unigram_counts[token] >= 2 else "UNK" for token in tokens]

      # Second pass: Count n-grams
      for tokens in processed_reviews:
          for n in range(self.ngram_size):
              for i in range(len(tokens) - n):
                  ngram = tuple(tokens[i:i + n + 1]) if n > 0 else tokens[i]  # Use tuple for bigram and above
                  self.ngram_count[n][ngram] += 1

    def add_k_smooth_prob(self, n_minus1_gram, unigram, k):
      ngram = n_minus1_gram + (unigram,)

      ngram_count = self.ngram_count[len(n_minus1_gram)].get(ngram, 0)  # Count of n-gram
      context_count = self.ngram_count[len(n_minus1_gram) - 1].get(n_minus1_gram, 0)  # Count of context

      # Compute vocabulary size (excluding <s>)
      vocabulary_size = len([word for word in self.ngram_count[0].keys() if word != self.bos_token])

      probability = (ngram_count + k) / (context_count + k * vocabulary_size) if context_count > 0 else 0

      return probability

Now test ```linear_interp_prob``` on the character-level model.







In [9]:
test_data = ['plan', 'plant', 'planet']
k = 0.5
test_char_lm = NGramLM(bos_token, eos_token, char_tokenizer, 4)
test_char_lm.get_ngram_counts(test_data)
print(f"Vocabulary: {list(test_char_lm.ngram_count[0].keys())}")
# test
trigram = ('p', 'l', 'a')
unigram = 'n'
print(f"Probability of seen: {test_char_lm.add_k_smooth_prob(trigram, unigram, k)}")

trigram = ('p', 'l', 'a')
unigram = 't'
print(f"Probability of unseen: {test_char_lm.add_k_smooth_prob(trigram, unigram, k)}")

Vocabulary: ['<s>', 'p', 'l', 'a', 'n', '</s>', 't', 'UNK']
Probability of seen: 0.5384615384615384
Probability of unseen: 0.07692307692307693


### C.1.3 Linear interpolation [4 points]
Similarly, **fill in** the function ```linear_interp_prob``` so that given a bigram $(w_1, w_2)$, a unigram $w_3$, and list of values [$\lambda_1$, $\lambda_2$, $\lambda_3$], return $p(w_3|(w_1, w_2))$ after applying linear interpolation.

Once again, implement this flexibly enough to operate on four-grams for the character-level model.

A couple notes:
- In certain instances, you may end up with a fraction 0/0 due to the fact that the count for both the n-gram and (n-1)-gram are equal to zero. In these instances, treat the entire fraction as 0 (even though technically $\frac{0}{0} \neq 0$).
- `<s>` should **NOT** be included when counting the total number of tokens for the unigram probability.

In [10]:
bos_token = "<s>"
eos_token = "</s>"

class NGramLM:
    """N-gram language model."""

    def __init__(self, bos_token, eos_token, tokenizer, ngram_size):
        self.ngram_count = {i: defaultdict(int) for i in range(ngram_size)}
        self.ngram_size = ngram_size
        self.vocab_sum = None  # Could be useful in linear interpolation
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.tokenizer = tokenizer

    def get_ngram_counts(self, reviews):
      unigram_counts = defaultdict(int)

      processed_reviews = []
      for review in reviews:
          tokens = self.tokenizer(review.lower())  # Tokenize and lowercase
          tokens = [self.bos_token] * (self.ngram_size - 1) + tokens + [self.eos_token]  # Add BOS/EOS
          processed_reviews.append(tokens)

          # Count unigrams
          for token in tokens:
              unigram_counts[token] += 1

      # Replace rare unigrams (<2 occurrences) with "UNK"
      for i, tokens in enumerate(processed_reviews):
          processed_reviews[i] = [token if unigram_counts[token] >= 2 else "UNK" for token in tokens]

      # Second pass: Count n-grams
      for tokens in processed_reviews:
          for n in range(self.ngram_size):
              for i in range(len(tokens) - n):
                  ngram = tuple(tokens[i:i + n + 1]) if n > 0 else tokens[i]  # Use tuple for bigram and above
                  self.ngram_count[n][ngram] += 1

    def add_k_smooth_prob(self, n_minus1_gram, unigram, k):
      ngram = n_minus1_gram + (unigram,)

      ngram_count = self.ngram_count[len(n_minus1_gram)].get(ngram, 0)  # Count of n-gram
      context_count = self.ngram_count[len(n_minus1_gram) - 1].get(n_minus1_gram, 0)  # Count of context

      # Compute vocabulary size (excluding <s>)
      vocabulary_size = len([word for word in self.ngram_count[0].keys() if word != self.bos_token])

      probability = (ngram_count + k) / (context_count + k * vocabulary_size) if context_count > 0 else 0

      return probability

    def linear_interp_prob(self, n_minus1_gram, unigram, lambdas):
      assert len(lambdas) == len(n_minus1_gram) + 1, "Lambda count must match n-gram order"

      probability = 0
      for i in range(len(lambdas)):
          if i == 0:  # Unigram probability
              count_ngram = self.ngram_count[0].get(unigram, 0)
              total_count = sum(self.ngram_count[0].values()) - self.ngram_count[0].get(self.bos_token, 0)
          else:  # Higher n-grams
              ngram = tuple(n_minus1_gram[-i:]) + (unigram,)
              count_ngram = self.ngram_count[i].get(ngram, 0)
              total_count = self.ngram_count[i - 1].get(tuple(n_minus1_gram[-i:]), 0)

          level_probability = (count_ngram / total_count) if total_count > 0 else 0
          probability += lambdas[i] * level_probability

      return probability

In [11]:
lambda1 = 0.6
lambda2 = 0.2
lambda3 = 0.1
lambda4 = 0.1
lambdas = [lambda1, lambda2, lambda3, lambda4]

trigram = ('p', 'l', 'a')
unigram = 'n'
print(f"Probability of seen: {test_char_lm.linear_interp_prob(trigram, unigram, lambdas)}")

trigram = ('p', 'l', 'a')
unigram = 't'
print(f"Probability of unseen: {test_char_lm.linear_interp_prob(trigram, unigram, lambdas)}")


AttributeError: 'NGramLM' object has no attribute 'linear_interp_prob'

### C.1.4 Calculate next word probability [2 points]
**Fill in** the function ```get_probability``` that calculates $p(w_3|(w_1, w_2))$ using either add-k smoothing or linear interpolation that you implemented above. The input is a dictionary that specifies how should you do the smoothing. Once again, program this function flexibly so that it works in the character-level model as well.

In [ ]:
class NGramLM:
    """N-gram language model."""

    def __init__(self, bos_token, eos_token, tokenizer, ngram_size):
        self.ngram_count = {i: defaultdict(int) for i in range(ngram_size)}
        self.ngram_size = ngram_size
        self.vocab_sum = None  # Useful for smoothing
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.tokenizer = tokenizer

    def get_ngram_counts(self, reviews):
        """Processes text and collects n-gram counts up to ngram_size."""
        unigram_counts = defaultdict(int)

        # First pass: Tokenize, lowercase, add BOS/EOS, and count unigrams
        processed_reviews = []
        for review in reviews:
            tokens = self.tokenizer(review.lower())  # Tokenize and lowercase
            tokens = [self.bos_token] * (self.ngram_size - 1) + tokens + [self.eos_token]  # Add BOS/EOS
            processed_reviews.append(tokens)

            # Count unigrams
            for token in tokens:
                unigram_counts[token] += 1

        # Replace rare unigrams (<2 occurrences) with "UNK"
        for i, tokens in enumerate(processed_reviews):
            processed_reviews[i] = [token if unigram_counts[token] >= 2 else "UNK" for token in tokens]

        # Second pass: Count n-grams
        for tokens in processed_reviews:
            for n in range(self.ngram_size):
                for i in range(len(tokens) - n):
                    ngram = tuple(tokens[i:i + n + 1]) if n > 0 else tokens[i]  # Use tuple for bigram and above
                    self.ngram_count[n][ngram] += 1

    def linear_interp_prob(self, n_minus1_gram, unigram, lambdas):
        """
        Compute probability using linear interpolation.
        Supports both word and character models.

        Args:
            n_minus1_gram: The preceding (n-1)-gram (tuple).
            unigram: The next token to predict.
            lambdas: List of lambda weights.

        Returns:
            Interpolated probability.
        """
        assert len(lambdas) == len(n_minus1_gram) + 1, "Lambda count must match n-gram order"

        probability = 0

        # Iterate over all n-gram levels (unigram, bigram, trigram, etc.)
        for i in range(len(lambdas)):
            if i == 0:  # Unigram probability
                count_ngram = self.ngram_count[0].get(unigram, 0)
                total_count = sum(self.ngram_count[0].values()) - self.ngram_count[0].get(self.bos_token, 0)
            else:  # Higher n-grams (bigram, trigram, etc.)
                ngram = tuple(n_minus1_gram[-i:]) + (unigram,)
                count_ngram = self.ngram_count[i].get(ngram, 0)
                total_count = self.ngram_count[i - 1].get(tuple(n_minus1_gram[-i:]), 0)

            # Avoid 0/0 errors
            level_probability = (count_ngram / total_count) if total_count > 0 else 0

            # Apply lambda weight
            probability += lambdas[i] * level_probability

        return probability

    def get_probability(self, n_minus1_gram, unigram, smoothing_args):
      """
      Compute the probability of `unigram` given `n_minus1_gram` using the specified smoothing method.

      Args:
          n_minus1_gram: The preceding (n-1)-gram (tuple).
          unigram: The next token to predict.
          smoothing_args: A dictionary specifying the smoothing method and its parameters.

      Returns:
          Probability of `unigram` given `n_minus1_gram`.
      """
      method = smoothing_args.get('method', 'add_k')  # Default to add-k

      if method == 'add_k':
          k = smoothing_args.get('k', 1.0)  # Default k value if not provided
          return self.add_k_smooth_prob(n_minus1_gram, unigram, k)

      elif method == 'linear':
          lambdas = smoothing_args.get('lambdas', [0.6, 0.2, 0.2])  # Default lambda values
          return self.linear_interp_prob(n_minus1_gram, unigram, lambdas)

      else:
          raise ValueError("Invalid smoothing method. Use 'add_k' or 'linear'.")


### C.1.5 Calculate perplexity [4 points]
One way to evaluate the language model is to calculate its perplexity on some validation data. **Fill in** the function `get_perplexity` so that, given a document and smoothing arguments, it returns the perplexity of the document. Remember to follow the **same** processing steps you used previously. To avoid numerical underflow, calculate the log of the perplexity first i.e.

$$
PPL(W) = \exp\left(\log\left(\sqrt[N]{\prod_{i=1}^N{\frac{1}{p(w_i | w_{i-n+1} \dots w_{i-1})}}}\right)\right) = \exp\left(-\frac{1}{N} \sum_{i=1}^N \log{p(w_i | w_{i-n+1} \dots w_{i-1})}\right)
$$

where $n$ is the n-gram order (e.g., $n=3$ for the trigram word-level model or $n=4$ for the 4-gram character-level model).  
In particular for a trigram model, $w_0 = w_{-1} = \text{<s>}$ and $w_N = \text{</s>}$. Remember to program this flexibly enough that it can work for both the word-level and character-level models.

In [ ]:
class NGramLM:
    """N-gram language model."""

    def __init__(self, bos_token, eos_token, tokenizer, ngram_size):
        self.ngram_count = {i: defaultdict(int) for i in range(ngram_size)}
        self.ngram_size = ngram_size
        self.vocab_sum = None  # Useful for smoothing
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.tokenizer = tokenizer

    def get_ngram_counts(self, reviews):
        """Processes text and collects n-gram counts up to ngram_size."""
        unigram_counts = defaultdict(int)

        # First pass: Tokenize, lowercase, add BOS/EOS, and count unigrams
        processed_reviews = []
        for review in reviews:
            tokens = self.tokenizer(review.lower())  # Tokenize and lowercase
            tokens = [self.bos_token] * (self.ngram_size - 1) + tokens + [self.eos_token]  # Add BOS/EOS
            processed_reviews.append(tokens)

            # Count unigrams
            for token in tokens:
                unigram_counts[token] += 1

        # Replace rare unigrams (<2 occurrences) with "UNK"
        for i, tokens in enumerate(processed_reviews):
            processed_reviews[i] = [token if unigram_counts[token] >= 2 else "UNK" for token in tokens]

        # Second pass: Count n-grams
        for tokens in processed_reviews:
            for n in range(self.ngram_size):
                for i in range(len(tokens) - n):
                    ngram = tuple(tokens[i:i + n + 1]) if n > 0 else tokens[i]  # Use tuple for bigram and above
                    self.ngram_count[n][ngram] += 1

    def linear_interp_prob(self, n_minus1_gram, unigram, lambdas):
        """
        Compute probability using linear interpolation.
        Supports both word and character models.

        Args:
            n_minus1_gram: The preceding (n-1)-gram (tuple).
            unigram: The next token to predict.
            lambdas: List of lambda weights.

        Returns:
            Interpolated probability.
        """
        assert len(lambdas) == len(n_minus1_gram) + 1, "Lambda count must match n-gram order"

        probability = 0

        # Iterate over all n-gram levels (unigram, bigram, trigram, etc.)
        for i in range(len(lambdas)):
            if i == 0:  # Unigram probability
                count_ngram = self.ngram_count[0].get(unigram, 0)
                total_count = sum(self.ngram_count[0].values()) - self.ngram_count[0].get(self.bos_token, 0)
            else:  # Higher n-grams (bigram, trigram, etc.)
                ngram = tuple(n_minus1_gram[-i:]) + (unigram,)
                count_ngram = self.ngram_count[i].get(ngram, 0)
                total_count = self.ngram_count[i - 1].get(tuple(n_minus1_gram[-i:]), 0)

            # Avoid 0/0 errors
            level_probability = (count_ngram / total_count) if total_count > 0 else 0

            # Apply lambda weight
            probability += lambdas[i] * level_probability

        return probability

    def get_probability(self, n_minus1_gram, unigram, smoothing_args):
      """
      Compute the probability of `unigram` given `n_minus1_gram` using the specified smoothing method.

      Args:
          n_minus1_gram: The preceding (n-1)-gram (tuple).
          unigram: The next token to predict.
          smoothing_args: A dictionary specifying the smoothing method and its parameters.

      Returns:
          Probability of `unigram` given `n_minus1_gram`.
      """
      method = smoothing_args.get('method', 'add_k')  # Default to add-k

      if method == 'add_k':
          k = smoothing_args.get('k', 1.0)  # Default k value if not provided
          return self.add_k_smooth_prob(n_minus1_gram, unigram, k)

      elif method == 'linear':
          lambdas = smoothing_args.get('lambdas', [0.6, 0.2, 0.2])  # Default lambda values
          return self.linear_interp_prob(n_minus1_gram, unigram, lambdas)

      else:
          raise ValueError("Invalid smoothing method. Use 'add_k' or 'linear'.")



    def get_perplexity(self, text, smoothing_args):
      """
      Compute the perplexity of a given text using the specified smoothing method.

      Args:
          text: The input text (string) to evaluate.
          smoothing_args: A dictionary specifying the smoothing method and its parameters.

      Returns:
          Perplexity score (float).
      """
      # Tokenize and preprocess the text
      tokens = self.tokenizer(text.lower())  # Tokenization and lowercasing
      tokens = [self.bos_token] * (self.ngram_size - 1) + tokens + [self.eos_token]  # Add BOS and EOS

      log_prob_sum = 0
      N = len(tokens) - (self.ngram_size - 1)  # Number of valid predictions

      # Iterate over the text using n-grams
      for i in range(self.ngram_size - 1, len(tokens)):
          n_minus1_gram = tuple(tokens[i - (self.ngram_size - 1):i])  # Context (n-1) words
          unigram = tokens[i]  # Next word/character

          # Get probability using smoothing method
          prob = self.get_probability(n_minus1_gram, unigram, smoothing_args)

          # Handle log(0) case: if prob is 0, return high perplexity
          if prob == 0:
              return float('inf')

          log_prob_sum += np.log(prob)

      # Compute perplexity
      perplexity = np.exp(-log_prob_sum / N)

      return perplexity


### C.1.6 Search hyperparameters [6 points]
Now you are ready to find your best language models! First find the best k value for the word-level model and for the character-level model using add-k smoothing. You need to search k in this list: \[0.2, 0.4, 0.6, 0.8, 1.0\].

**Fill in** the function ```search_k``` such that given a validation set, return the best k value on it. Print out the perplexity (average on the whole validation set) for each k.

In [ ]:
def search_k(self, dev_data):
    """
    Search for the best k value for add-k smoothing by testing on the validation set.

    Args:
        dev_data: Validation dataset (list of text samples).

    Returns:
        Best k value (float) that minimizes perplexity.
    """
    k_values = [0.2, 0.4, 0.6, 0.8, 1.0]
    best_k = None
    lowest_perplexity = float('inf')

    print("Searching for best k...")

    for k in k_values:
        smoothing_args = {'method': 'add_k', 'k': k}
        total_perplexity = 0
        count = 0

        for text in dev_data:
            perplexity = self.get_perplexity(text, smoothing_args)
            total_perplexity += perplexity
            count += 1

        avg_perplexity = total_perplexity / count
        print(f"k={k}: Perplexity={avg_perplexity:.3f}")

        if avg_perplexity < lowest_perplexity:
            lowest_perplexity = avg_perplexity
            best_k = k

    return best_k


Similarly, **fill in** the function ```search_lambda``` such that, given a validation set, returns the best $\lambda$ values on it. You need to choose the search list by yourself. Print out the best set of $\lambda$ and corresponding perplexity. To get full credits, your perplexity scores need to be < 180 for the word-level model and < 15 for the character-level model.

Note: this code block might take a couple minutes to run.

In [ ]:
import numpy as np
from collections import defaultdict

class NGramLM:
    """N-gram language model."""

    def __init__(self, bos_token, eos_token, tokenizer, ngram_size):
        """
        Initialize the NGram Language Model.
        """
        self.ngram_count = {}
        for i in range(ngram_size):
            self.ngram_count[i] = defaultdict(int)

        self.ngram_size = ngram_size
        self.vocab_sum = None  # Could be useful in linear interpolation
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.tokenizer = tokenizer
        self.unigram_count = defaultdict(int)
        self.vocab_size = 0

    def get_ngram_counts(self, reviews):
        """
        Store counts in self.ngram_count.
        """
        processed_reviews = []

        for review in reviews:
            tokens = self.tokenizer(review.lower())

            # ✅ Add correct BOS padding
            bos_padding = [self.bos_token] * (self.ngram_size - 1)
            tokens = bos_padding + tokens + [self.eos_token]

            processed_reviews.append(tokens)

        # ✅ Step 1: Count unigrams first
        unigram_temp = defaultdict(int)
        for review in processed_reviews:
            for token in review:
                unigram_temp[token] += 1

        # ✅ Step 2: Replace low-frequency words with "UNK"
        for review in processed_reviews:
            for i in range(len(review)):
                if unigram_temp[review[i]] < 2:
                    review[i] = "UNK"

        # ✅ Step 3: Update final unigram counts
        self.unigram_count.clear()
        for review in processed_reviews:
            for token in review:
                self.unigram_count[token] += 1

        # ✅ Step 4: Count n-grams properly
        for review in processed_reviews:
            for n in range(1, self.ngram_size + 1):
                for i in range(len(review) - n + 1):
                    ngram = tuple(review[i:i + n])
                    self.ngram_count[n - 1][ngram] += 1

        # ✅ Step 5: Compute vocabulary size
        self.vocab_size = len(self.unigram_count) - (1 if self.bos_token in self.unigram_count else 0)

    def add_k_smooth_prob(self, n_minus1_gram, unigram, k):
        """
        Compute probability using Add-K smoothing.
        """
        probability = 0
        ngram = n_minus1_gram + (unigram,)
        prefix_count = self.ngram_count[len(n_minus1_gram)].get(n_minus1_gram, 0)
        ngram_count = self.ngram_count[len(n_minus1_gram)].get(ngram, 0)

        # ✅ Prevent division by zero
        probability = (ngram_count + k) / (prefix_count + k * self.vocab_size + 1)
        return probability

    def linear_interp_prob(self, ngram_prefix, next_token, lambdas):
        """
        Compute probability using Linear Interpolation Smoothing.
        """
        total_prob = 0

        # ✅ Prevent division by zero
        unigram_prob = (self.unigram_count.get(next_token, 0) + 1) / (self.vocab_size + 1)
        total_prob += lambdas[0] * unigram_prob

        if len(ngram_prefix) >= 1:
            bigram_count = self.ngram_count[1].get((ngram_prefix[-1], next_token), 0)
            bigram_prefix_count = self.unigram_count.get(ngram_prefix[-1], 0)
            bigram_prob = (bigram_count + 1) / (bigram_prefix_count + self.vocab_size + 1)
            total_prob += lambdas[1] * bigram_prob

        if len(ngram_prefix) >= 2:
            trigram_count = self.ngram_count[2].get((ngram_prefix[-2], ngram_prefix[-1], next_token), 0)
            trigram_prefix_count = self.ngram_count[1].get((ngram_prefix[-2], ngram_prefix[-1]), 0)
            trigram_prob = (trigram_count + 1) / (trigram_prefix_count + self.vocab_size + 1)
            total_prob += lambdas[2] * trigram_prob

        return total_prob

    def get_probability(self, ngram_prefix, next_token, smoothing_args):
        method = smoothing_args.get('method', 'add_k')

        if method == 'add_k':
            k = smoothing_args.get('k', 0.5)
            return self.add_k_smooth_prob(ngram_prefix, next_token, k)
        elif method == 'linear':
            lambdas = smoothing_args.get('lambdas', [0.6, 0.2, 0.2])
            return self.linear_interp_prob(ngram_prefix, next_token, lambdas)
        else:
            raise ValueError("Unknown smoothing method. Use 'add_k' or 'linear'.")

    def get_perplexity(self, text, smoothing_args):
        tokens = self.tokenizer(text.lower())
        bos_padding = [self.bos_token] * (self.ngram_size - 1)
        tokens = bos_padding + tokens + [self.eos_token]
        log_prob_sum = 0
        N = len(tokens) - (self.ngram_size - 1)

        for i in range(self.ngram_size - 1, len(tokens)):
            ngram_prefix = tuple(tokens[i - (self.ngram_size - 1):i])
            next_token = tokens[i]
            prob = self.get_probability(ngram_prefix, next_token, smoothing_args)

            if prob > 0:
                log_prob_sum += np.log(prob)
            else:
                log_prob_sum += np.log(1e-10)
        avg_log_prob = log_prob_sum / N
        perplexity = np.exp(-avg_log_prob)
        return perplexity

    def search_k(self, dev_text):
        k_values = [0.2, 0.4, 0.6, 0.8, 1.0]
        best_k = None
        min_perplexity = float('inf')
        print("Evaluating different k values...")

        for k in k_values:
            smoothing_args = {'method': 'add_k', 'k': k}
            total_perplexity = 0
            num_sentences = len(dev_text)

            for sentence in dev_text:
                total_perplexity += self.get_perplexity(sentence, smoothing_args)
            avg_perplexity = total_perplexity / num_sentences
            print(f"k = {k}: Perplexity = {avg_perplexity}")

            if avg_perplexity < min_perplexity:
                min_perplexity = avg_perplexity
                best_k = k
        return best_k

    def search_lambda(self, dev_text):
        lambda_options = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
        best_lambdas = None
        min_perplexity = float('inf')
        lambda_combinations = [
            (l1, l2, round(1 - l1 - l2, 2))
            for l1, l2 in itertools.product(lambda_options, repeat=2)
            if l1 + l2 <= 1
        ]
        print("Searching best lambda...")
        for lambdas in lambda_combinations:
            smoothing_args = {'method': 'linear', 'lambdas': lambdas}
            total_perplexity = np.mean([self.get_perplexity(sent, smoothing_args) for sent in dev_text])

            print(f"Lambdas = {lambdas}: Perplexity = {total_perplexity}")
            if total_perplexity < min_perplexity:
                min_perplexity = total_perplexity
                best_lambdas = lambdas
        return best_lambdas

# Example Test
dev_text = ["This is a validation sentence.", "The model should predict well.", "Perplexity should be low."]
word_lm = NGramLM("<s>", "</s>", str.split, 3)
char_lm = NGramLM("<s>", "</s>", list, 4)
print("Word LM")
best_k_word = word_lm.search_k(dev_text)
best_lambda_word = word_lm.search_lambda(dev_text)
print("Char LM")
best_k_char = char_lm.search_k(dev_text)
best_lambda_char = char_lm.search_lambda(dev_text)


Word LM
Evaluating different k values...
k = 0.2: Perplexity = 4.999999999999999
k = 0.4: Perplexity = 2.5
k = 0.6: Perplexity = 1.6666666666666667
k = 0.8: Perplexity = 1.25
k = 1.0: Perplexity = 1.0
Searching best lambda...
Lambdas = (0.1, 0.1, 0.8): Perplexity = 1.0
Lambdas = (0.1, 0.2, 0.7): Perplexity = 1.0
Lambdas = (0.1, 0.3, 0.6): Perplexity = 1.0
Lambdas = (0.1, 0.4, 0.5): Perplexity = 1.0
Lambdas = (0.1, 0.5, 0.4): Perplexity = 1.0
Lambdas = (0.1, 0.6, 0.3): Perplexity = 1.0
Lambdas = (0.2, 0.1, 0.7): Perplexity = 1.0
Lambdas = (0.2, 0.2, 0.6): Perplexity = 1.0
Lambdas = (0.2, 0.3, 0.5): Perplexity = 1.0
Lambdas = (0.2, 0.4, 0.4): Perplexity = 1.0
Lambdas = (0.2, 0.5, 0.3): Perplexity = 1.0
Lambdas = (0.2, 0.6, 0.2): Perplexity = 1.0
Lambdas = (0.3, 0.1, 0.6): Perplexity = 1.0
Lambdas = (0.3, 0.2, 0.5): Perplexity = 1.0
Lambdas = (0.3, 0.3, 0.4): Perplexity = 1.0
Lambdas = (0.3, 0.4, 0.3): Perplexity = 1.0
Lambdas = (0.3, 0.5, 0.2): Perplexity = 1.0
Lambdas = (0.3, 0.6, 0.1):

### C.1.7 Generate reviews [5 points]
Finally, you can automatically generate text using your language models. **Fill in** the function ```generate_text``` to generate a sentence based on an input prompt. To generate the text, you need to find the distribution of the next word given previous two words (or next character given the previous three characters). Then you need to **randomly** sample the next word/character based on the distribution (i.e., do **NOT** use greedy decoding, which is deterministically choosing the most probable unigram). After the word/character is sampled, append it to the current text and continue generating the next word/character. You need to repeat this process untill the current sequence **has 15 words/characters** (including prompts) or you **generate the &lt;/s&gt; token**.

Note that there exist more advanced methods to generate text from language models such as beam search, top-k sampling, and top-p sampling. You can refer to [this blog](https://huggingface.co/blog/how-to-generate) to get an idea of what they are. In this assignment, you are not required to implement the advanced methods. Simply sampling from the trigram/four-gram distribution is good enough.

Begin with the word-level model here:

In [ ]:
def generate_text(self, prompt, smoothing_args):
    generated = list(prompt)
    max_length = 15
    print(f"Prompt: {' '.join(generated)}")

    while len(generated) < max_length:
        ngram_prefix = tuple(generated[-(self.ngram_size - 1):])
        next_word_probs = {}
        for token in self.unigram_count.keys():
            if token != self.bos_token:
                prob = self.get_probability(ngram_prefix, token, smoothing_args)
                if prob > 0:
                    next_word_probs[token] = prob
        if not next_word_probs:
            break
        tokens, probs = zip(*next_word_probs.items())
        next_token = random.choices(tokens, weights=probs, k=1)[0]

        if next_token == self.eos_token:
            break
        generated.append(next_token)
    print(f"Generated Text: {' '.join(generated)}\n")
setattr(NGramLM, "generate_text", generate_text)

# Example Test
prompts = [['The', 'location'], ['We', 'ate'], ['I', 'thought'], ['It', 'had']]
smoothing_args1 = {'method': 'add_k', 'k': 0.5}
print("Word LM")
for prompt in prompts:
    word_lm.generate_text(prompt, smoothing_args1)

Word LM
Prompt: The location
Generated Text: The location

Prompt: We ate
Generated Text: We ate

Prompt: I thought
Generated Text: I thought

Prompt: It had
Generated Text: It had



Now test the character-level model:

In [ ]:
prompts = ['The store', 'We ate']
for prompt in prompts:
    print("Char LM")
    prompt_tokenized = char_tokenizer(prompt)
    char_lm.generate_text(prompt_tokenized, smoothing_args1)

Char LM
Prompt: T h e   s t o r e
Generated Text: T h e   s t o r e

Char LM
Prompt: W e   a t e
Generated Text: W e   a t e



### C.1.8 Predict the class of a review [4 points]
The Yelp review dataset contains several columns specifying attributes of each review:
- ```stars``` indicates on a scale from 1 to 5 how many stars the reviewer gave in the review
- ```cool```, ```useful```, and ```funny``` all indicate how many users rated the review as cool, useful or funny

Using the same techniques developed above, this section requires you to do the following:
- Fill in the function ```load_new_data``` to generate a split in the data. For instance, one class could be reviews with at least one funny rating and the opposing class could be reviews not marked as funny by anyone. In this function, use the techniques at the beginning of the notebook to split the overall dataset into two classes of your choice
- Fill in the function ```predict_class``` to predict the class to which the review in ```test_review.txt``` belongs. This function will take in two word-level language models (each trained on the training data for one class) and compute the probability of the review generated by each language model. I.e., $p(W|LM_1)$ and $p(W|LM_2)$, or equivalently $PPL(W)$ based on $LM_1$ and $LM_2$. Print out the perplexity of each language model and your prediction.

The test review used has the following attributes:
- ```stars``` : 2
- ```useful``` : 3
- ```funny``` : 7
- ```cool``` : 3

In [ ]:
def load_new_data(df):
    class1_data = df[df['funny'] > 0]['text'].tolist()
    class2_data = df[df['funny'] == 0]['text'].tolist()
    return class1_data, class2_data

def predict_class(test_review_file, class1_lm, class2_lm, smoothing_args):
    with open(test_review_file, 'r') as file:
        test_review = file.read().strip()
    perplexity_class1 = class1_lm.get_perplexity(test_review, smoothing_args)
    perplexity_class2 = class2_lm.get_perplexity(test_review, smoothing_args)
    print(f"Perplexity for Class 1 (Funny Reviews): {perplexity_class1}")
    print(f"Perplexity for Class 2 (Not Funny Reviews): {perplexity_class2}")
    predicted_class = "Class 1 (Funny)" if perplexity_class1 < perplexity_class2 else "Class 2 (Not Funny)"
    print(f"Predicted Class: {predicted_class}")

filename = 'yelp.csv'
df = pd.read_csv(filename)
class1_data, class2_data = load_new_data(df)
class1_lm = NGramLM("<s>", "</s>", word_tokenize, 3)
class1_lm.get_ngram_counts(class1_data)
class2_lm = NGramLM("<s>", "</s>", word_tokenize, 3)
class2_lm.get_ngram_counts(class2_data)
print("Finding best lambda values for Word LM...")
word_lambda = class1_lm.search_lambda(class1_data + class2_data)
smoothing_args = {'method': 'linear', 'lambdas': word_lambda}
predict_class('test_review.txt', class1_lm, class2_lm, smoothing_args)

Finding best lambda values for Word LM...
Searching best lambda...
Lambdas = (0.1, 0.1, 0.8): Perplexity = 2709.6141272781924
Lambdas = (0.1, 0.2, 0.7): Perplexity = 2241.2141745937893
Lambdas = (0.1, 0.3, 0.6): Perplexity = 2037.5233246516361
Lambdas = (0.1, 0.4, 0.5): Perplexity = 1919.654327721313
Lambdas = (0.1, 0.5, 0.4): Perplexity = 1841.4605085950789
Lambdas = (0.1, 0.6, 0.3): Perplexity = 1785.191440602496
Lambdas = (0.2, 0.1, 0.7): Perplexity = 1959.4417292725914
Lambdas = (0.2, 0.2, 0.6): Perplexity = 1635.0371230473554
Lambdas = (0.2, 0.3, 0.5): Perplexity = 1485.267817612301
Lambdas = (0.2, 0.4, 0.4): Perplexity = 1396.036036808305
Lambdas = (0.2, 0.5, 0.3): Perplexity = 1335.7886127230472
Lambdas = (0.2, 0.6, 0.2): Perplexity = 1291.9188993898329
Lambdas = (0.3, 0.1, 0.6): Perplexity = 1603.0764480535083
Lambdas = (0.3, 0.2, 0.5): Perplexity = 1352.1879152544252
Lambdas = (0.3, 0.3, 0.4): Perplexity = 1231.2367696667939
Lambdas = (0.3, 0.4, 0.3): Perplexity = 1157.5269311

## C.2 Naive Bayes for Text Classification [26 points]
In this problem, you will build naive bayes classifiers to do text classification. You will use the clickbait headlines dataset, which contains examples of legitimate news headlines and clickbait news headlines. The original dataset can be found in [this GitHub repository](https://github.com/bhargaviparanjape/clickbait) and [this paper](https://arxiv.org/abs/1610.09786).
### C.2.1 Load dataset [4 points]
To get started, **fill in** the function ```load_headlines``` to load the clickbait dataset into pandas dataframes. The file ```clickbait_data.csv``` contains a partially processed subset of the data. It contains two columns: (1) ```is_clickbait``` is 1 when the row contains a clickbait headline and 0 when it doesn't and (2) ```text```, which contains the headline itself.

To get started, **fill in** the function ```load_headlines``` to load the clickbait dataset into a pandas dataframe. To do this, you will need to do the following:

1. Read in the ```text``` and ```is_clickbait``` columns.
2. Rename the ```is_clickbait``` column to ```label```

In [ ]:
def load_headlines(filename):
    df = pd.read_csv(filename, usecols=["is_clickbait", "text"])
    df.rename(columns={"is_clickbait": "label"}, inplace=True)
    return df
all_data = load_headlines('clickbait_data.csv')
train, test = train_test_split(all_data, train_size=0.9, random_state=42)
display(train.head())
display(test.head())

,label,text
4896,1,Here's How To Wrap A Present Like An Actual Adult
4782,1,58 French Villages That Should Be On Your Buck...
1496,1,We Need To Talk About Gigi Hadid For Like 10 S...
1957,1,Rihanna Released A New Song But The Internet C...
9171,0,300 Laid Off at Sears


,label,text
6252,0,"'Denmark will be attacked' says one expert, 'D..."
4684,1,14 Things You Never Noticed During The Opening...
1731,1,This Is What It's Actually Like To Live As An ...
4742,1,This Is What Happens When A Lesbian Runs A Gay...
4521,1,25 Gifts For The Ultimate Wine Lover In Your Life


### C.2.2 Dataset statistics [3 points]
Before start training classifiers, you need to calculate some basic statistics of the dataset. **Fill in** the function ```get_basic_stats``` to print out the following statistics of the training data:
- Average number of tokens per headline
- Standard deviation of the number of tokens per headline
- Total number of legitimate headlines
- Total number of clickbait headlines

Note: you can use any tokenization method you like.

In [ ]:
def get_basic_stats(df):
    token_lengths = df["text"].apply(lambda x: len(word_tokenize(x)))
    avg_len = np.mean(token_lengths)
    std_len = np.std(token_lengths)
    num_articles = df["label"].value_counts().to_dict()
    print(f"Average number of tokens per headline: {avg_len:.2f}")
    print(f"Standard deviation: {std_len:.2f}")
    print(f"Number of legitimate/clickbait headlines: {num_articles}")
get_basic_stats(train)

Average number of tokens per headline: 9.63
Standard deviation: 2.96
Number of legitimate/clickbait headlines: {1: 4508, 0: 4492}


### C.2.3 Data processing and ngram calculation [6 points]
Now you need to calculate the ngram counts. **Fill in** the function ```fit``` that, given a dataframe of training data, calculates the ngram counts in each category and the prior probability for each category. Concretely, **store** the total occurrence of each ngram in each category in a list called ```self.ngram_count``` so that ```self.ngram_count[0]``` contains $count(w, c_0)$ for all $w$ in the vocabulary, and ```self.ngram_count[1]``` contains $count(w, c_1)$, etc. ```self.ngram_count[i]``` should be an array of shape $(1,|V|)$, where $V$ is the vocabulary (total vocabulary across both classes). **Store** the total occurrence of all ngrams in each category in a list called ```self.total_count``` so that ```self.total_count[0]``` $=\sum_{w\in V}count(w, c_0)$, and ```self.total_count[1]``` $=\sum_{w\in V}count(w, c_1)$, etc. **Store** the prior probability for each category in ```self.category_prob```. You need to follow these rules when calculating the counts:
- convert all letters to lowercase;
- include both unigrams and bigrams;
- ignore terms that appear in more than 80\% of the headlines;
- ignore terms that appear in less than 3 headlines.

Hint: use ```CountVectorizer``` in sklearn and store it as ```self.vectorizer```. You need to use **both legitimate and clickbait headlines** to get the vocabulary.

In [ ]:
class NaiveBayes:
    def __init__(self):
        self.vectorizer = None
        self.ngram_count = []
        self.total_count = []
        self.category_prob = []

    def fit(self, data):
        texts = data["text"].str.lower().tolist()
        labels = data["label"].tolist()
        self.vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.8)
        X = self.vectorizer.fit_transform(texts)
        X_clickbait = X[np.array(labels) == 1]
        X_legit = X[np.array(labels) == 0]
        self.ngram_count = [X_legit.sum(axis=0), X_clickbait.sum(axis=0)]
        self.total_count = [self.ngram_count[0].sum(), self.ngram_count[1].sum()]
        num_samples = len(labels)
        self.category_prob = [
            sum(1 for label in labels if label == 0) / num_samples,  # P(legit)
            sum(1 for label in labels if label == 1) / num_samples   # P(clickbait)
        ]
naive_bayes = NaiveBayes()
naive_bayes.fit(train)
print(f"Probability for each category: {naive_bayes.category_prob}")
print(f"Length of self.ngram_count: {len(naive_bayes.ngram_count)}")
print(f"Shape of the counts for 1st category: {naive_bayes.ngram_count[0].shape}")
print(f"Number of non-zero terms for 1st category: {(naive_bayes.ngram_count[0] > 0).sum()}")
print(f"Maximum count of the 1st category: {naive_bayes.ngram_count[0].max()}")
print(f"Minimum count of the 1st category: {naive_bayes.ngram_count[0].min()}")
print(f"Sum of ngram count for 1st category: {naive_bayes.ngram_count[0].sum()}")
print(f"Total count for each category: {naive_bayes.total_count}")


Probability for each category: [0.4991111111111111, 0.5008888888888889]
Length of self.ngram_count: 2
Shape of the counts for 1st category: (1, 7208)
Number of non-zero terms for 1st category: 4746
Maximum count of the 1st category: 1296
Minimum count of the 1st category: 0
Sum of ngram count for 1st category: 35547
Total count for each category: [35547, 56388]


### C.2.4 Calculate posterior probability for a category [4 points]
Next, you will use the vectorizer and ngram counts to calculate the posterior probability of a category. In this homework, we have two categories: legitimate and clickbait. **Fill in** the function ```calculate_prob``` that given a list of articles $docs$, a category index $i$, return $\log\left(p(c_i)p(d|c_i)\right)=\log\left(p(c_i)\prod_{x\in X}p(x|c_i)\right)$ for each article $d$ in $docs$, where $X$ is the set of unigrams and bigrams in **both** article $d$ and vocabulary $V$.

- Use **add-one smoothing** in your calculation.
- Simply discard unseen unigrams/bigrams (do not use add-one smoothing to account for them).
- Calculate the **sum of logarithms** to avoid issues with underflow.

In [ ]:
class NaiveBayes:
    def __init__(self):
        self.vectorizer = None
        self.ngram_count = []
        self.total_count = []
        self.category_prob = []

    def fit(self, data):
        texts = data["text"].str.lower().tolist()
        labels = data["label"].tolist()
        self.vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.8)
        X = self.vectorizer.fit_transform(texts)
        X_clickbait = X[np.array(labels) == 1]
        X_legit = X[np.array(labels) == 0]
        self.ngram_count = [X_legit.sum(axis=0), X_clickbait.sum(axis=0)]
        self.total_count = [self.ngram_count[0].sum(), self.ngram_count[1].sum()]
        num_samples = len(labels)
        self.category_prob = [
            sum(1 for label in labels if label == 0) / num_samples,
            sum(1 for label in labels if label == 1) / num_samples
        ]

    def calculate_prob(self, docs, c_i):
        X_docs = self.vectorizer.transform(docs)
        ngram_counts = self.ngram_count[c_i]
        total_count = self.total_count[c_i]
        vocab_size = len(self.vectorizer.vocabulary_)
        log_probs = []
        log_prior = np.log(self.category_prob[c_i])

        for i in range(X_docs.shape[0]):
            log_likelihood = 0
            for j in range(X_docs.shape[1]):
                count_x = X_docs[i, j]
                if count_x > 0:
                    word_count = ngram_counts[0, j]
                    prob_x_given_ci = (word_count + 1) / (total_count + vocab_size)
                    log_likelihood += count_x * np.log(prob_x_given_ci)
            log_probs.append(log_prior + log_likelihood)
        return log_probs
naive_bayes = NaiveBayes()
naive_bayes.fit(train)
test_docs = ["United Kingdom officially exits the European Union",
             "How to Lose a Guy in 10 Days"]
prob1 = naive_bayes.calculate_prob(test_docs, 0)
prob2 = naive_bayes.calculate_prob(test_docs, 1)
print(f"Probability for category 0 (Legit): {prob1}")
print(f"Probability for category 1 (Clickbait): {prob2}")

Probability for category 0 (Legit): [-61.981371323938866, -69.61324807840032]
Probability for category 1 (Clickbait): [-77.83310517298293, -66.98539561864114]


### C.2.5 Predict labels for new headlines [2 points]
With the posterior probability of each category, you can predict the label for new headlines. **Fill in** the function ```predict``` that, given a list of headlines, returns the predicted categories of the headlines.

In [ ]:
class NaiveBayes:
    def __init__(self):
        self.vectorizer = None
        self.ngram_count = []
        self.total_count = []
        self.category_prob = []

    def fit(self, data):
        texts = data["text"].str.lower().tolist()
        labels = data["label"].tolist()
        self.vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.8)
        X = self.vectorizer.fit_transform(texts)
        X_clickbait = X[np.array(labels) == 1]
        X_legit = X[np.array(labels) == 0]
        self.ngram_count = [X_legit.sum(axis=0), X_clickbait.sum(axis=0)]
        self.total_count = [self.ngram_count[0].sum(), self.ngram_count[1].sum()]
        num_samples = len(labels)
        self.category_prob = [
            sum(1 for label in labels if label == 0) / num_samples,
            sum(1 for label in labels if label == 1) / num_samples
        ]

    def calculate_prob(self, docs, c_i):
        X_docs = self.vectorizer.transform(docs)
        ngram_counts = self.ngram_count[c_i]
        total_count = self.total_count[c_i]
        vocab_size = len(self.vectorizer.vocabulary_)
        log_probs = []
        log_prior = np.log(self.category_prob[c_i])
        for i in range(X_docs.shape[0]):
            log_likelihood = 0
            for j in range(X_docs.shape[1]):
                count_x = X_docs[i, j]
                if count_x > 0:
                    word_count = ngram_counts[0, j]
                    prob_x_given_ci = (word_count + 1) / (total_count + vocab_size)
                    log_likelihood += count_x * np.log(prob_x_given_ci)
            log_probs.append(log_prior + log_likelihood)
        return log_probs

    def predict(self, docs):
        prob_legit = self.calculate_prob(docs, 0)
        prob_clickbait = self.calculate_prob(docs, 1)
        predictions = [1 if prob_clickbait[i] > prob_legit[i] else 0 for i in range(len(docs))]
        return predictions
naive_bayes = NaiveBayes()
naive_bayes.fit(train)
test_docs = ["United Kingdom officially exits the European Union",
             "How to Lose a Guy in 10 Days"]
preds = naive_bayes.predict(test_docs)
print(f"Prediction: {preds}")

Prediction: [0, 1]


### C.2.6 Calculate evaluation metrics [5 points]
To evaluate a classifier, you need to calculate some evaluation metrics. **Fill in** the function ```evaluate``` that, given a list of predictions and a list of true labels, returns the accuracy, macro f1-score, and micro f1-score. You can **NOT** use functions in sklearn.

In [ ]:
def evaluate(predictions, labels):
    correct = sum(1 for p, l in zip(predictions, labels) if p == l)
    accuracy = correct / len(labels)
    class_counts = {0: {"TP": 0, "FP": 0, "FN": 0}, 1: {"TP": 0, "FP": 0, "FN": 0}}
    for p, l in zip(predictions, labels):
        if p == l:
            class_counts[l]["TP"] += 1
        else:
            class_counts[p]["FP"] += 1
            class_counts[l]["FN"] += 1
    macro_f1 = 0
    for cls in [0, 1]:
        TP, FP, FN = class_counts[cls]["TP"], class_counts[cls]["FP"], class_counts[cls]["FN"]
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        macro_f1 += f1
    macro_f1 /= 2
    total_TP = class_counts[0]["TP"] + class_counts[1]["TP"]
    total_FP = class_counts[0]["FP"] + class_counts[1]["FP"]
    total_FN = class_counts[0]["FN"] + class_counts[1]["FN"]
    micro_f1 = (2 * total_TP) / (2 * total_TP + total_FP + total_FN) if (2 * total_TP + total_FP + total_FN) > 0 else 0
    return accuracy, macro_f1, micro_f1
predictions = [1, 1, 0, 1, 0, 0, 1]
labels = [1, 0, 0, 1, 0, 1, 1]
accuracy, mac_f1, mic_f1 = evaluate(predictions, labels)
print(f"Accuracy: {accuracy:.2f}")
print(f"Macro F1-score: {mac_f1:.2f}")
print(f"Micro F1-score: {mic_f1:.2f}")

Accuracy: 0.71
Macro F1-score: 0.71
Micro F1-score: 0.71


### C.2.7 Test classifier on test data [2 points]
Finally, you are ready to evaluate your classifier on the test data! Run the following cell to make predictions and print out performance.

In [ ]:
predictions = naive_bayes.predict(test.text.tolist())
labels = test.label.tolist()
accuracy, mac_f1, mic_f1 = evaluate(predictions, labels)
print(f"Accuracy: {accuracy}")
print(f"Macro f1: {mac_f1}")
print(f"Micro f1: {mic_f1}")

Accuracy: 0.964
Macro f1: 0.9639994239907839
Micro f1: 0.964
